In [1]:

import torch
import clip
from PIL import Image
import os
import numpy as np
from torch.nn.functional import cosine_similarity

# 1. Setting up the environment 
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 2. Uploading references from folders (each folder is a color type)
reference_folder = "clustered_by_color_type"
reference_embeddings = []
reference_labels = []

for label in os.listdir(reference_folder):
    label_path = os.path.join(reference_folder, label)
    if not os.path.isdir(label_path):
        continue
    for file in os.listdir(label_path):
        if file.endswith((".jpg", ".jpeg")):
            image_path = os.path.join(label_path, file)
            if not os.path.isfile(image_path): 
                print(f"Skipped missing file: {image_path}")
                continue
            try:
                image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
                with torch.no_grad():
                    emb = model.encode_image(image)
                reference_embeddings.append(emb / emb.norm(dim=-1, keepdim=True))
                reference_labels.append(label)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")


# 3. Downloading all images from the tests folder
test_folder = "tests"
test_images = [f for f in os.listdir(test_folder) if f.endswith((".jpg", ".jpeg"))]

# 4. Type prediction for each image
print("\n Predicted types for images in 'tests':\n")

for test_img_name in test_images:
    test_image_path = os.path.join(test_folder, test_img_name)
    test_image = preprocess(Image.open(test_image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        test_embedding = model.encode_image(test_image)
    test_embedding = test_embedding / test_embedding.norm(dim=-1, keepdim=True)

    similarities = [cosine_similarity(test_embedding, ref_emb).item() for ref_emb in reference_embeddings]
    best_index = int(np.argmax(similarities))
    predicted_type = reference_labels[best_index]

    print(f" {test_img_name}: {predicted_type.capitalize()}")



 Predicted types for images in 'tests':

 test1.jpg: Spring
 test10.jpg: Winter
 test11.jpg: Spring
 test12.jpeg: Winter
 test13.jpeg: Winter
 test14.jpeg: Autumn
 test15.jpeg: Autumn
 test16.jpeg: Spring
 test17.jpeg: Autumn
 test18.jpeg: Autumn
 test19.jpeg: Winter
 test2.jpg: Spring
 test20.jpeg: Spring
 test21.jpeg: Spring
 test22.jpeg: Spring
 test23.jpeg: Autumn
 test24.jpeg: Summer
 test25.jpeg: Autumn
 test26.jpeg: Autumn
 test27.jpeg: Spring
 test28.jpeg: Spring
 test29.jpeg: Autumn
 test3.jpg: Winter
 test30.jpeg: Spring
 test31.jpeg: Winter
 test32.jpeg: Spring
 test33.jpeg: Autumn
 test34.jpeg: Spring
 test35.jpeg: Spring
 test36.jpeg: Spring
 test37.jpeg: Autumn
 test38.jpeg: Spring
 test39.jpeg: Spring
 test4.jpg: Spring
 test40.jpeg: Spring
 test41.jpeg: Winter
 test42.jpeg: Autumn
 test43.jpeg: Winter
 test44.jpeg: Winter
 test45.jpeg: Spring
 test46.jpeg: Spring
 test47.jpeg: Winter
 test48.jpeg: Spring
 test49.jpeg: Spring
 test5.jpg: Winter
 test50.jpeg: Autumn
 tes

In [2]:
import pandas as pd
from sklearn.metrics import classification_report

# Загружаем реальные метки
df_true = pd.read_csv("true_labels.csv")

# Предсказанные метки, собранные в цикле
predicted = []

for test_img_name in test_images:
    test_image_path = os.path.join(test_folder, test_img_name)
    test_image = preprocess(Image.open(test_image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        test_embedding = model.encode_image(test_image)
    test_embedding = test_embedding / test_embedding.norm(dim=-1, keepdim=True)

    similarities = [cosine_similarity(test_embedding, ref_emb).item() for ref_emb in reference_embeddings]
    best_index = int(np.argmax(similarities))
    predicted_type = reference_labels[best_index]

    predicted.append((test_img_name, predicted_type))

# Создаём DataFrame с предсказаниями
df_pred = pd.DataFrame(predicted, columns=["filename", "predicted_type"])

# Объединяем с true labels
df_merged = pd.merge(df_true, df_pred, on="filename")

# Оцениваем точность
print("\n📊 Classification Report:\n")
print(classification_report(df_merged["color_type"], df_merged["predicted_type"]))



📊 Classification Report:

              precision    recall  f1-score   support

      autumn       0.50      0.43      0.46        21
      spring       0.32      0.71      0.44        21
      summer       0.25      0.04      0.07        24
      winter       0.39      0.33      0.36        21

    accuracy                           0.37        87
   macro avg       0.36      0.38      0.33        87
weighted avg       0.36      0.37      0.32        87

